In [1]:
import os
os.chdir('..')

In [4]:
import pandas as pd

df = pd.read_parquet('s3://kishan.murthy/thumbnail_scoring/BGE_embeddings_100k.parquet')

In [6]:
# import pandas as pd
# from psycopg2.pool import ThreadedConnectionPool
# from psycopg2.extras import execute_values
# from typing import List, Dict, Any
# import threading
# from queue import Queue
# from tqdm import tqdm
# from threading import Event

# class TitleEmbeddingsLoader:
#     def __init__(
#         self,
#         df: pd.DataFrame,
#         db_config,
#         db_workers: int = 4,
#         batch_size: int = 1000,
#         queue_size: int = 2000
#     ):
#         self.df = df
#         self.db_workers = db_workers
#         self.batch_size = batch_size
        
#         # Initialize queue
#         self.db_queue = Queue(maxsize=queue_size)
        
#         # Initialize database connection pool
#         self.db_pool = ThreadedConnectionPool(
#             minconn=2,
#             maxconn=db_workers + 1,
#             host=db_config.host,
#             port=db_config.port,
#             database=db_config.database,
#             user=db_config.user,
#             password=db_config.password
#         )
        
#         # Control flags
#         self.stop_event = Event()
#         self.error_event = Event()
        
#         # Progress tracking
#         self.total_items = len(df)
#         self.processed_items = 0
#         self.progress_lock = threading.Lock()
#         self.pbar = None
    
#     def _db_worker(self):
#         """Worker function to insert embeddings into database"""
#         conn = self.db_pool.getconn()
#         batch = []
        
#         try:
#             while not self.stop_event.is_set():
#                 try:
#                     item = self.db_queue.get(timeout=1)
#                     if item is None:
#                         break
                    
#                     batch.append((item['video_id'], item['embedding'].tolist(), None))
                    
#                     if len(batch) >= self.batch_size:
#                         self._process_batch(conn, batch)
#                         batch = []
                    
#                     self.db_queue.task_done()
                    
#                 except Queue.Empty:
#                     if batch:  # Process remaining items
#                         self._process_batch(conn, batch)
#                         batch = []
#                     continue
                    
#         except Exception as e:
#             print(f"DB worker error: {str(e)}")
#             self.error_event.set()
#         finally:
#             if batch:
#                 try:
#                     self._process_batch(conn, batch)
#                 except Exception as e:
#                     print(f"Error in final batch processing: {str(e)}")
#             self.db_pool.putconn(conn)

#     def _process_batch(self, conn, batch):
#         """Process a batch of items"""
#         if not batch:
#             return
            
#         with conn.cursor() as cur:
#             execute_values(
#                 cur,
#                 """
#                 INSERT INTO video_embeddings 
#                 (video_id, title_embedding, thumbnail_embedding)
#                 VALUES %s
#                 ON CONFLICT (video_id) DO UPDATE
#                 SET title_embedding = EXCLUDED.title_embedding
#                 """,
#                 batch
#             )
#             conn.commit()
        
#         with self.progress_lock:
#             self.processed_items += len(batch)
#             if self.pbar is not None:
#                 self.pbar.update(len(batch))
    
#     def populate_embeddings(self):
#         """Main function to populate title embeddings"""
#         try:
#             print(f"Processing {self.total_items} videos")
#             self.pbar = tqdm(total=self.total_items, desc="Processing")
            
#             # Start DB workers
#             db_threads = []
#             for _ in range(self.db_workers):
#                 t = threading.Thread(target=self._db_worker)
#                 t.daemon = True
#                 t.start()
#                 db_threads.append(t)
            
#             # Feed DB queue with all data first
#             for _, row in self.df.iterrows():
#                 if self.error_event.is_set():
#                     break
#                 self.db_queue.put(row, timeout=30)
            
#             # Signal workers to stop after all data is queued
#             for _ in range(self.db_workers):
#                 self.db_queue.put(None)
            
#             # Wait for queue to empty
#             self.db_queue.join()
            
#             # Set stop event and cleanup
#             self.stop_event.set()
#             for t in db_threads:
#                 t.join(timeout=1)
            
#         except Exception as e:
#             print(f"Error in populate_embeddings: {str(e)}")
#             self.error_event.set()
            
#         finally:
#             self.stop_event.set()
#             if hasattr(self, 'db_pool'):
#                 self.db_pool.closeall()
#             if self.pbar is not None:
#                 self.pbar.close()
            
#             if self.error_event.is_set():
#                 print("\nProcessing stopped due to errors")
#             else:
#                 print(f"\nProcessed {self.processed_items} videos successfully")


In [7]:
from utils.db import DatabaseConfig
from utils.embedding_loader import TitleEmbeddingsLoader

config = DatabaseConfig()
loader = TitleEmbeddingsLoader(
    df=df,
    db_config=config,
    db_workers=4,
    batch_size=1000,
    queue_size=2000
)

loader.populate_embeddings()

Processing 100000 videos


Processing: 100%|██████████| 100000/100000 [02:30<00:00, 1016.04it/s]

: 